# Predict "Responsiveness" in the Control Group

To ensure that the effect we find can be associated with the intervention and not other factors of reduced drinking, we assign participants from the control randomly to "on" and "off" weeks to check if we can make predictions about their reduced drinking.

In [ ]:
# Standard libraries
import os
import csv
import time
import warnings
from copy import deepcopy
import joblib 
from itertools import product, combinations, chain

# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import shap

# Statistical analysis
from scipy import stats
from scipy.stats import norm, mannwhitneyu

# Machine learning models
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Preprocessing and model evaluation
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, roc_auc_score, f1_score, recall_score,
    precision_recall_curve, confusion_matrix, auc, precision_score,
    balanced_accuracy_score, matthews_corrcoef
)

# Model interpretation
from sklearn.inspection import PartialDependenceDisplay

# Progress bar
from tqdm import tqdm

# Suppress warnings
warnings.filterwarnings('ignore')

## Pre-Processing

### Define threshold for responsiveness

Indicate change threshold that qualifies a participant as responsive vs non-responsive

In [ ]:
# DEFINE RESPONSIVENESS
# avg reduction in drinking occasions between active and control weeks
def_response_drink_occasions = -1

### Load data

In [ ]:
data_study1 = pd.read_csv('../data/intervention_time/osf_study1.csv')
data_study2 = pd.read_csv('../data/intervention_time/osf_study2.csv')

# Study 1 baseline data (train/val input)
b1_alcohol_self = pd.read_csv('../data/baseline/alcoholself_bucket280225.csv', index_col=0)
b2_group_subjective = pd.read_csv('../data/baseline/subjective_grouperceptions_280225.csv', index_col=0)
b3_group_sociometric = pd.read_csv('../data/baseline/data_social.csv')
b4_brain = pd.read_csv('../data/baseline/brain_bucket_280225.csv', index_col=0)
b5_demographic = pd.read_csv('../data/baseline/demographic_bucket280225.csv', index_col=0)
b6_psychometric = pd.read_csv('../data/baseline/psychometrics_bucket280225.csv', index_col=0)

# Study 2 subjective data (test input)
b2_group_subjective_study2 = pd.read_csv('../data/baseline/subjective_grouperceptions_test.csv')

# Sample characteristics
baseline_demo_study2 = pd.read_csv('../data/baseline/demo_study2_full.csv')
baseline_demo_study1 = pd.read_csv('../data/baseline/demo_study1_full.csv')
baseline_auq_study1 = pd.read_csv('../data/baseline/study_1_all_drinking.csv')

# Study 1 & 2 drinking/responsiveness data (output -> prediction target)
if def_response_drink_occasions == -1:
    responsive_study1 = pd.read_csv('../data/intervention_time/responsiveness_study1.csv', index_col=0).reset_index()
elif def_response_drink_occasions == -0.5:
    responsive_study1 = pd.read_csv('../data/intervention_time/responsiveness_study1_-0_5.csv', index_col=0).reset_index()
elif def_response_drink_occasions == -2:
    responsive_study1 = pd.read_csv('../data/intervention_time/responsiveness_study1_-2.csv', index_col=0).reset_index()

responsive_study2 = pd.read_csv('../data/intervention_time/responsiveness_study2.csv', index_col=0).reset_index()

# All subjective group perceptions
b2_group_subjective = pd.read_csv('../data/baseline/subjective_grouperceptions_all.csv', index_col=0)

control_ppl = pd.read_csv('../data/added_analysis/weekly_drinking_summary_controlgroup.csv', index_col=0).reset_index()


In [ ]:
control_ppl

## Sample Characteristics
### Overall

In [ ]:
active_study1_ids = pd.read_csv("../data/study_1_ids.csv")
active_study2_ids = pd.read_csv("../data/study_2_ids.csv")

control_summary_df = pd.read_csv("../data/intervention_time/intervention_time_controls_study1and2.csv")
summary_stats = control_summary_df.agg({
    'adherence': ['mean', 'std', 'min', 'max'],
    'drinking_occasions': ['mean', 'std', 'min', 'max'],
    'avg_drinks_per_occasion': ['mean', 'std', 'min', 'max']
}).round(2)
summary_stats

Occasions: 1.46, 1.52, (0.0-6.25)
Amount: 2.04, 1.86, (0.0-8.67)

### Study 1

In [ ]:
study1_control_subset = control_summary_df[control_summary_df['id'].isin(data_study1['id'])]
study1_control_ids = study1_control_subset["id"]

summary_stats = study1_control_subset.agg({
    'adherence': ['mean', 'std', 'min', 'max'],
    'drinking_occasions': ['mean', 'std', 'min', 'max'],
    'avg_drinks_per_occasion': ['mean', 'std', 'min', 'max']
}).round(2)
summary_stats

Occasions: 2.2, 1.59, (0.0-6.25)
Amount: 3.6, 1.86, (0.0-8.67)

In [ ]:
study2_control_subset = control_summary_df[control_summary_df['id'].isin(data_study2['id'])]
study2_control_subset

In [ ]:
study1_control_ids

In [ ]:
# Get AUQ baselines

baseline_subset = (
    baseline_auq_study1
    .loc[:, ['SID', 'AUQ_drink_amount', 'AUQ_drink_frequency']]
    .rename(columns={'SID': 'id'})
)

baseline_subset = baseline_subset[baseline_subset['id'].isin(data_study1['id'])]

active_df = baseline_subset[baseline_subset['id'].isin(active_study1_ids['id'])]
control_df = baseline_subset[
    (~baseline_subset['id'].isin(active_study1_ids['id'])) &
    (baseline_subset['id'].isin(study1_control_ids))
]


In [ ]:
active_df

In [ ]:
control_df

In [ ]:
# Descriptive stats
desc = {}
for group_name, df in {'active': active_df, 'control': control_df}.items():
    desc[group_name] = df[['AUQ_drink_amount', 'AUQ_drink_frequency']].agg(['mean', 'std', 'min', 'max']).round(2)

desc_table = pd.concat(desc, axis=1)
print(desc_table)

# Statistical tests
for var in ['AUQ_drink_amount', 'AUQ_drink_frequency']:
    t, p = stats.ttest_ind(active_df[var], control_df[var], nan_policy='omit', equal_var=False)
    print(f"{var}: t = {t:.2f}, p = {p:.3f}")

In [ ]:
baseline_auq_study1

### Demographics

In [ ]:
# gender = 1 is Male
baseline_demo_study1 = (
    baseline_demo_study1[['pID', 'age', 'gender_numeric', 'race_numeric']]
    .rename(columns={'pID': 'id'}, errors='ignore')
)

In [ ]:
baseline_demo_study1 = baseline_demo_study1[baseline_demo_study1['id'].isin(data_study1['id'])]

active_df = baseline_demo_study1[baseline_demo_study1['id'].isin(active_study1_ids['id'])]
control_df = baseline_demo_study1[
    (~baseline_demo_study1['id'].isin(active_study1_ids['id'])) &
    (baseline_demo_study1['id'].isin(study1_control_ids))
]

In [ ]:
def describe_group(df, name):
    print(f"\n===== {name} group =====")
    # Age
    age_mean = df['age'].mean()
    age_sd = df['age'].std()
    age_min = df['age'].min()
    age_max = df['age'].max()
    print(f"Age: M = {round(age_mean, 2)}, SD = {round(age_sd, 2)}, Min = {age_min}, Max = {age_max}")

    # Gender distribution
    gender_counts = df['gender_numeric'].value_counts(dropna=False).sort_index()
    gender_percent = round(100 * gender_counts / gender_counts.sum(), 1)
    print("Gender distribution:")
    for g, c in gender_counts.items():
        label = "Missing" if pd.isna(g) else f"Gender {g}"
        print(f"  {label}: {c} ({gender_percent[g]}%)")

    # Race distribution
    race_counts = df['race_numeric'].value_counts(dropna=False).sort_index()
    race_percent = round(100 * race_counts / race_counts.sum(), 1)
    print("Race distribution:")
    for r, c in race_counts.items():
        label = "Missing" if pd.isna(r) else f"Race {r}"
        print(f"  {label}: {c} ({race_percent[r]}%)")

# Descriptives for each group
describe_group(active_df, "Active")
describe_group(control_df, "Control")

# ---- Statistical comparisons ----
print("\n===== Statistical comparisons =====")

# Continuous variable: Age (Welch t-test)
t_age, p_age = stats.ttest_ind(
    active_df['age'], control_df['age'],
    nan_policy='omit', equal_var=False
)
print(f"Age: t = {t_age:.2f}, p = {p_age:.3f}")

# Categorical variable: Gender (Chi-square)
gender_table = pd.crosstab(
    baseline_demo_study1['gender_numeric'],
    baseline_demo_study1['id'].isin(active_study1_ids['id'])
)
chi2_g, p_g, dof_g, _ = stats.chi2_contingency(gender_table)
print(f"Gender: χ²({dof_g}) = {chi2_g:.2f}, p = {p_g:.3f}")

# Categorical variable: Race (Chi-square)
race_table = pd.crosstab(
    baseline_demo_study1['race_numeric'],
    baseline_demo_study1['id'].isin(active_study1_ids['id'])
)
chi2_r, p_r, dof_r, _ = stats.chi2_contingency(race_table)
print(f"Race: χ²({dof_r}) = {chi2_r:.2f}, p = {p_r:.3f}")

### Study 2

In [ ]:
study2_control_subset = control_summary_df[control_summary_df['id'].isin(data_study2['id'])]
study2_control_ids = study2_control_subset["id"]

summary_stats = study2_control_subset.agg({
    'adherence': ['mean', 'std', 'min', 'max'],
    'drinking_occasions': ['mean', 'std', 'min', 'max'],
    'avg_drinks_per_occasion': ['mean', 'std', 'min', 'max']
}).round(2)
summary_stats

Occasions: 1.07, 1.34, (0.0-5.25)
Amount: 1.23, 1.25, (0.0-5.0)

## Dummy coding weeks as active or control WITHIN THE CONTROL GROUP - calculate 'responsiveness' when not under treatment

In [ ]:
# Define active/control weeks for each condition
ACAC_active_weeks = [1, 3]
ACAC_control_weeks = [2, 4]

CACA_active_weeks = [2, 4]
CACA_control_weeks = [1, 3]

def compute_responsiveness(df, active_weeks, control_weeks):
    active_avg = df[df['week'].isin(active_weeks)].groupby('id')['drinking_occasions'].mean()
    control_avg = df[df['week'].isin(control_weeks)].groupby('id')['drinking_occasions'].mean()

    comparison = pd.DataFrame({'active_avg': active_avg, 'control_avg': control_avg})
    comparison['responsive'] = (comparison['active_avg'] - comparison['control_avg'] <= -1).astype(int)
    return comparison.reset_index()

# Apply for both conditions
condition_ACAC = compute_responsiveness(control_ppl, ACAC_active_weeks, ACAC_control_weeks)
condition_CACA = compute_responsiveness(control_ppl, CACA_active_weeks, CACA_control_weeks)

In [ ]:
condition_ACAC
condition_CACA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Compute difference and descriptive stats
for name, df in {'ACAC': condition_ACAC, 'CACA': condition_CACA}.items():
    df['diff'] = df['control_avg'] - df['active_avg']
    mean_diff = df['diff'].mean()
    std_diff = df['diff'].std()
    print(f"{name} — Mean Δ(control−active): {mean_diff:.2f}, SD: {std_diff:.2f}")

# Combine both for plotting
plot_df = (
    pd.concat([
        condition_ACAC.assign(condition='ACAC'),
        condition_CACA.assign(condition='CACA')
    ])
)

# Plot
sns.set(style='whitegrid', palette='muted')
plt.figure(figsize=(6, 5))

ax = sns.boxplot(
    data=plot_df,
    x='condition',
    y='diff',
    width=0.4,
    showfliers=False,
    boxprops=dict(alpha=0.6)
)
sns.swarmplot(
    data=plot_df,
    x='condition',
    y='diff',
    color='black',
    alpha=0.6,
    size=4
)

# Reference and threshold lines
plt.axhline(0, color='black', linestyle='-', lw=1)
for thresh in [-0.5, -1.0]:
    plt.axhline(y=thresh, color='grey', linestyle='--', linewidth=1)
    plt.text(1.05, thresh + 0.05, f'{thresh}', color='grey', fontsize=9)

plt.title('Difference in Drinking Occasions (Active − Control)', fontsize=14)
plt.ylabel('Δ Control − Active (drinking occasions/week)')
plt.xlabel('Condition')
plt.tight_layout()
plt.show()



In [ ]:
print(condition_ACAC.responsive.sum())
len(condition_ACAC)

In [ ]:
print(condition_CACA.responsive.sum())
len(condition_CACA)

In [ ]:
condition_CACA

In [ ]:
EXCLUDE_VARS = ['active_avg', 'control_avg']

condition_ACAC.drop(columns=EXCLUDE_VARS, inplace=True)
condition_CACA.drop(columns=EXCLUDE_VARS, inplace=True)

Replace with `CONDITION_TO_TEST = condition_CACA` to test "off-on-off-on"

In [ ]:
CONDITION_TO_TEST = condition_ACAC

In [ ]:
# Stratified split based on the 'responsive' column
responsive_study1, responsive_study2 = train_test_split(
    CONDITION_TO_TEST,
    test_size=0.3,
    stratify=CONDITION_TO_TEST['responsive'],
    random_state=42
)

### Merge Baseline and Target Data

In [ ]:
# Training datasets -> Study 1
b2_group_subjective_response = pd.merge(b2_group_subjective, responsive_study1, on='id', how='inner')

print(f'Total IDs Study 1: {len(b2_group_subjective_response)}')
print(f'Responsive IDs Study 1: {b2_group_subjective_response[b2_group_subjective_response["responsive"] == 1]["id"].nunique()}')
print('----------')

# Testing dataset -> Study 2
b2_group_subjective_test = pd.merge(b2_group_subjective, responsive_study2, on='id', how='inner')
print(f'Total IDs Study 2: {len(b2_group_subjective_test)}')
print(f'Responsive IDs Study 2: {b2_group_subjective_test[b2_group_subjective_test["responsive"] == 1]["id"].nunique()}')

In [ ]:
dataframes = {
    'group_sub': b2_group_subjective_response
}

for key, df in dataframes.items():
    print(f"Missing values in '{key}':")
    print(df.isna().sum())
    print()  # for spacing between outputs

# Feature Selection

## Find highly correlated features within buckets
Find redundancy in features if they are highly correlated

In [ ]:
b2_group_subjective_response.columns

In [ ]:
TARGET_VAR = 'responsive'

In [ ]:
def find_highly_correlated_features(dataframes, threshold=0.8):
    """
    Identifies pairs of highly correlated features in each dataframe.
    :param dataframes: dict of {name: dataframe}
    :param threshold: correlation threshold to consider as "high"
    :return: dict of {name: list of correlated feature pairs}
    """
    correlated_features = {}
    for name, df in dataframes.items():
        # Exclude COMMON_VARS from the correlation computation
        columns_to_correlate = [col for col in df.columns if col != TARGET_VAR and col !='id']
        
        # Compute correlation matrix only for selected columns
        corr_matrix = df[columns_to_correlate].corr().abs()
        
        # Select the upper triangle of the correlation matrix
        upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        
        # Find pairs of features with correlation above the threshold
        correlated_pairs = [
            (col, idx, upper_triangle.loc[idx, col])
            for col in upper_triangle.columns
            for idx in upper_triangle.index
            if upper_triangle.loc[idx, col] > threshold
        ]
        
        # Store results for the current dataframe
        correlated_features[name] = correlated_pairs

    return correlated_features

In [ ]:
correlated_features = find_highly_correlated_features(dataframes, threshold=0.8)

# Display results
for name, pairs in correlated_features.items():
    print(f"\n{name} - Highly Correlated Features:")
    for col1, col2, corr_value in pairs:
        print(f"  {col1} ↔ {col2} : Correlation = {corr_value:.2f}")


In [ ]:
# Number of features per category
{key: df.shape[1] for key, df in dataframes.items()}

## Significance tests: Features
### Mann-Whitney U Tests

Hypothesis test for non-normally distributed data to check which of the remaining features show the most (significant) difference between the two groups (responsive vs non-responsive).

In [ ]:
def perform_mann_whitney_u(df, target_var, exclude_vars):
    results = {}
    for col in df.columns:
        if col not in exclude_vars and col != target_var:
            try:
                df[col] = pd.to_numeric(df[col], errors='coerce')
                group1 = df[df[target_var] == 0][col]
                group2 = df[df[target_var] == 1][col]
                stat, p_value = mannwhitneyu(group1, group2, alternative='two-sided')
                results[col] = {'U_statistic': stat, 'p_value': p_value}
            except Exception as e:
                results[col] = {'error': str(e)}
    return results

In [ ]:
mwu_results = {}
for name, df in dataframes.items():
    if name != 'demo' and TARGET_VAR in df.columns:
        mwu_results[name] = perform_mann_whitney_u(df, TARGET_VAR, 'id')

# Output summary
for name, results in mwu_results.items():
    print(f"\n{name} DataFrame Mann-Whitney U Test Results (p-value < 0.05):")
    
    # Ensure only variables with p-values < 0.05 are retained
    significant_results = {}
    for var, stats in results.items():
        if isinstance(stats, dict) and 'p_value' in stats and stats['p_value'] < 0.05:
            significant_results[var] = stats  
    
    if significant_results:
        df_significant = pd.DataFrame(significant_results).T  
        print(df_significant)
    else:
        print("No significant results (p-value < 0.05) found.")
    print("---------------")

# ML Models

## Train / Test Splits

In [ ]:

def prepare_features_and_targets(df, test_set=0, resampling=None):
    if TARGET_VAR not in df.columns:
        raise ValueError(f"Target variable '{TARGET_VAR}' not found in dataframe.")

    # Extract target variable and drop excluded columns
    targets = df[TARGET_VAR]
    features = df[[col for col in df.columns if col != TARGET_VAR and col != 'id']]
    features = features.drop(columns=[TARGET_VAR], errors='ignore')

    # Split into training and test sets (STRATIFIED)
    if test_set:
        X_train, X_test, Y_train, Y_test = train_test_split(
            features, targets, test_size=test_set, stratify=targets
        )
    else: 
        X_train = features
        Y_train = targets
        X_test = []
        Y_test = []

    # Median imputation for 'income_numeric' if it contains NA values
    if 'income_numeric' in X_train.columns:
        if X_train['income_numeric'].isna().any():
            X_train['income_numeric'].fillna(X_train['income_numeric'].median(), inplace=True)
        if isinstance(X_test, pd.DataFrame) and 'income_numeric' in X_test.columns and X_test['income_numeric'].isna().any():
            X_test['income_numeric'].fillna(X_test['income_numeric'].median(), inplace=True)

    if 'IAS_mean' in X_train.columns:
        if X_train['IAS_mean'].isna().any():
            X_train['IAS_mean'].fillna(X_train['IAS_mean'].median(), inplace=True)
        if isinstance(X_test, pd.DataFrame) and 'IAS_mean' in X_test.columns and X_test['IAS_mean'].isna().any():
            X_test['IAS_mean'].fillna(X_test['IAS_mean'].median(), inplace=True)

    # TODO: Handle all other missingness here
    return X_train, Y_train, X_test, Y_test

## Random Forest

In [ ]:
def random_forest_kfold_grid_search(
    X, Y, param_grid, k=5, CV_reps=1, eval_metric=['auc'], model_choice_metric='auc', 
    res_dir=".", model_type='rf', combo='alcohol'
):

    # Generate all parameter combinations
    param_combinations = list(product(*param_grid.values()))
    param_names = list(param_grid.keys())

    # Initialize variables to store the best model and scores
    best_model = None
    best_scores = None
    best_params = None
    best_model_choice_value = -np.inf  # Track the best model based on the chosen metric

    kf = StratifiedKFold(n_splits=k, shuffle=True)

    for params in param_combinations:
        current_params = dict(zip(param_names, params))

        # Store all fold results
        all_folds_metrics = {metric: [] for metric in eval_metric}

        for train_index, test_index in kf.split(X, Y):  # k-fold cv split

            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

            rep_metrics = {metric: [] for metric in eval_metric}  # Reset for each fold

            for _ in range(CV_reps):  # Repeat that split j times

                # Initialize the model with the current parameters
                if model_type == 'rf':
                    model = RandomForestClassifier(
                        n_estimators=current_params.get("n_estimators", 100),
                        max_depth=current_params.get("max_depth"),
                        min_samples_split=current_params.get("min_samples_split", 2),
                        min_samples_leaf=current_params.get("min_samples_leaf", 1),
                        class_weight="balanced"
                    )
                elif model_type == 'xgb':
                    model = XGBClassifier(
                        n_estimators=current_params.get("n_estimators", 100),
                        max_depth=current_params.get("max_depth", 6),
                        learning_rate=current_params.get("learning_rate", 0.1),
                        min_child_weight=current_params.get("min_child_weight", 1),
                        gamma=current_params.get("gamma", 0),
                        subsample=current_params.get("subsample", 1),
                        colsample_bytree=current_params.get("colsample_bytree", 1),
                        scale_pos_weight=current_params.get("scale_pos_weight", 1),
                        use_label_encoder=False,
                        eval_metric="logloss"
                    )

                model.fit(X_train, Y_train)
                Y_pred = model.predict(X_test)
                Y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None

                if 'auc' in eval_metric and Y_prob is not None:
                    rep_metrics['auc'].append(roc_auc_score(Y_test, Y_prob))
                if 'f1' in eval_metric:
                    rep_metrics['f1'].append(f1_score(Y_test, Y_pred))
                if 'accuracy' in eval_metric:
                    rep_metrics['accuracy'].append(accuracy_score(Y_test, Y_pred))
                if 'specificity' in eval_metric or 'sensitivity' in eval_metric:
                    tn, fp, fn, tp = confusion_matrix(Y_test, Y_pred).ravel()
                    specificity = tn / (tn + fp) if (tn + fp) > 0 else np.nan
                    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else np.nan
                    if 'specificity' in eval_metric:
                        rep_metrics['specificity'].append(specificity)
                    if 'sensitivity' in eval_metric:
                        rep_metrics['sensitivity'].append(sensitivity)
                if 'mcc' in eval_metric:
                    rep_metrics['mcc'].append(matthews_corrcoef(Y_test, Y_pred))
                if 'balancedAcc' in eval_metric:
                    rep_metrics['balancedAcc'].append(balanced_accuracy_score(Y_test, Y_pred))
                if 'pr_auc' in eval_metric and Y_prob is not None:
                    precision, recall, _ = precision_recall_curve(Y_test, Y_prob)
                    pr_auc = auc(recall, precision)
                    rep_metrics['pr_auc'].append(pr_auc)

            # Compute median scores per fold and store results
            fold_median_metrics = {metric: np.mean(values) for metric, values in rep_metrics.items()}
            for metric in eval_metric:
                all_folds_metrics[metric].append(fold_median_metrics[metric])

        # Compute final median scores over all folds
        median_rep_metrics = {metric: np.mean(values) for metric, values in all_folds_metrics.items()}

        # Select best model based on median of model_choice_metric
        if median_rep_metrics[model_choice_metric] > best_model_choice_value:
            best_model_choice_value = median_rep_metrics[model_choice_metric]
            best_model = model
            best_params = current_params
            best_scores = median_rep_metrics  # Store median scores for all metrics

    return best_model, best_scores, best_params


In [ ]:
def save_metrics_to_csv(results_dict, results_dir, filename):

    os.makedirs(results_dir, exist_ok=True)


    # Define the output file path
    file_path = os.path.join(results_dir, filename)
    
    # Extract all metric names
    all_metrics = set()
    for metrics in results_dict.values():
        all_metrics.update(metrics.keys())
    
    # Sort metrics for consistency
    all_metrics = sorted(all_metrics)

    # Open CSV file for writing
    with open(file_path, mode="w", newline="") as file:
        writer = csv.writer(file)
        
        # Write header
        header = ["run", "group"] + all_metrics
        writer.writerow(header)

        # Write data
        for group, metrics in results_dict.items():
            num_runs = len(next(iter(metrics.values())))  # Get number of runs from first metric
            for run_idx in range(num_runs):
                row = [run_idx, str(group)]  # Start with run index and group name
                for metric in all_metrics:
                    value = metrics.get(metric, [np.nan] * num_runs)[run_idx]  # Handle missing values
                    row.append(value)
                writer.writerow(row)

In [ ]:
def compute_test_metrics(Y_test, test_predictions, proba_predictions, test_scores):
    Y_test_flat = Y_test.ravel()
    
    test_scores['auc'].append(roc_auc_score(Y_test_flat, proba_predictions))
    test_scores['f1'].append(f1_score(Y_test_flat, test_predictions))
    test_scores['accuracy'].append(accuracy_score(Y_test_flat, test_predictions))

    tn, fp, fn, tp = confusion_matrix(Y_test_flat, test_predictions).ravel()
    test_scores['specificity'].append(tn / (tn + fp) if (tn + fp) > 0 else np.nan)
    test_scores['sensitivity'].append(tp / (tp + fn) if (tp + fn) > 0 else np.nan)
    test_scores['PPV'].append(tp / (tp + fp) if (tp + fp) > 0 else np.nan)
    test_scores['NPV'].append(tn / (tn + fn) if (tn + fn) > 0 else np.nan)
    test_scores['MCC'].append(matthews_corrcoef(Y_test_flat, test_predictions))
    test_scores['balancedAcc'].append(balanced_accuracy_score(Y_test_flat, test_predictions))

    precision, recall, _ = precision_recall_curve(Y_test_flat, proba_predictions)
    pr_auc = auc(recall, precision)
    test_scores['pr_auc'].append(pr_auc)

    test_scores['tn'].append(tn)
    test_scores['fp'].append(fp)
    test_scores['fn'].append(fn)
    test_scores['tp'].append(tp)

    return test_scores

In [ ]:
def flatten_score_dict(score_dict, res_dir=None, filename=None):
    rows = []
    for combination, metrics in score_dict.items():
        row = {"Combination": combination}
        for metric, values in metrics.items():
            row[f"{metric}_mean"] = values["mean"]
            row[f"{metric}_CI_lower"] = values["95%_CI"][0]
            row[f"{metric}_CI_upper"] = values["95%_CI"][1]
        rows.append(row)

    df = pd.DataFrame(rows)
    df_comb = pd.DataFrame(df["Combination"].tolist(), columns=[f"Factor_{i+1}" for i in range(df["Combination"].map(len).max())])
    df = pd.concat([df_comb, df.drop(columns="Combination")], axis=1)

    if res_dir and filename:
        df.to_csv(f"{res_dir}/{filename}", index=False)

    return df

In [ ]:
def plot_shap_summary_with_percentages(all_shap_values, all_test_data, res_dir, combo):
    # Combine SHAP values and test data
    final_shap_values = np.vstack(all_shap_values)
    final_test_data = pd.concat(all_test_data, ignore_index=True)

    # Compute relative importance
    mean_abs_shap = np.abs(final_shap_values).mean(axis=0)
    rel_importance = 100 * mean_abs_shap / mean_abs_shap.sum()

    # Plot SHAP summary without showing
    plt.figure()
    shap.summary_plot(final_shap_values, final_test_data, show=False, cmap='winter')

    # Get current axis and y-tick labels
    ax = plt.gca()
    feature_names = [tick.get_text() for tick in ax.get_yticklabels()]

    # Use Index.get_loc instead of list
    col_index = final_test_data.columns
    feature_order = [col_index.get_loc(name) for name in feature_names]

    # Add percentage values
    percent_labels = [f"{name} ({rel_importance[i]:.1f}%)" for name, i in zip(feature_names, feature_order)]
    ax.set_yticklabels(percent_labels, fontsize=10)

    # Save updated plot
    plt.tight_layout()
    plt.savefig(f"{res_dir}/{combo}_shap_summary_plot_with_percentages.png", dpi=300, bbox_inches="tight")
    plt.close()

    # Return top 2 most important features (by mean absolute SHAP)
    top2_indices = np.argsort(mean_abs_shap)[-2:][::-1]
    top2_features = final_test_data.columns[top2_indices].tolist()
    return top2_features


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
from scipy.interpolate import interp1d

def plot_pdp_across_runs(best_model, res_dir, all_test_data, feature_names=None, interaction_pair=None, colors=None, title=None):
    """
    Plots PDPs with mean and std across multiple test sets for each feature.
    Optionally adds an interaction plot.

    Parameters:
        best_model: trained model
        all_test_data: list of pd.DataFrames used for PDP evaluation
        feature_names: list of features to plot (default: all features in data)
        interaction_pair: tuple of two features to plot interaction PDP
        colors: optional color list
    """
    if colors is None:
        colors = ["#22223B", "#4A4E69", "#9A8C98", "#C9ADA7", "#F2E9E4"]

    final_test_data = pd.concat(all_test_data, ignore_index=True)

    if feature_names is None:
        feature_names = final_test_data.columns.tolist()

    num_features = len(feature_names)
    num_plots = num_features + (1 if interaction_pair else 0)
    num_cols = 3
    num_rows = -(-num_plots // num_cols)

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols * 5, num_rows * 4))
    axes = axes.flatten()

    for idx, feature_name in enumerate(feature_names):
        pdp_values = []
        feature_values_list = []

        for dat in all_test_data:
            ax_dummy = plt.figure().add_subplot()
            ax_dummy.set_visible(False)
            pdp_display = PartialDependenceDisplay.from_estimator(best_model, dat, [feature_name], ax=ax_dummy)
            plt.close(ax_dummy.figure)

            pdp_x = pdp_display.lines_[0][0].get_xdata()
            pdp_y = pdp_display.lines_[0][0].get_ydata()
            pdp_values.append(pdp_y)
            feature_values_list.append(pdp_x)

        common_feature_values = np.linspace(min(map(min, feature_values_list)),
                                            max(map(max, feature_values_list)), num=100)

        interpolated_pdp_values = []
        for i in range(len(pdp_values)):
            f_interp = interp1d(feature_values_list[i], pdp_values[i], kind="linear", fill_value="extrapolate")
            interpolated_pdp_values.append(f_interp(common_feature_values))

        pdp_values = np.array(interpolated_pdp_values)
        pdp_mean = np.mean(pdp_values, axis=0)
        pdp_std = np.std(pdp_values, axis=0)

        ax = axes[idx]
        ax.plot(common_feature_values, pdp_mean, label="Mean PDP", color=colors[0], lw=2)
        ax.fill_between(common_feature_values, pdp_mean - pdp_std, pdp_mean + pdp_std,
                        color=colors[2], alpha=0.5, label="Std Dev")
        ax.set_ylabel("Predicted Value")
        ax.set_title(f"PDP for {feature_name}")
        ax.legend()

    if interaction_pair:
        ax = axes[num_features]
        PartialDependenceDisplay.from_estimator(best_model, final_test_data,
                                                [interaction_pair], ax=ax)
        ax.set_title(f"Interaction: {interaction_pair[0]} & {interaction_pair[1]}")

    for i in range(num_plots, len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()

    # Build filename
    interaction_suffix = f"_{interaction_pair[0]}_{interaction_pair[1]}" if interaction_pair else ""
    if not title:
        filename = f"{res_dir}/pdp_plots{interaction_suffix}.png"
    else:
        filename = f"{res_dir}/pdp_plots{title}.png"
    # Save figure
    plt.savefig(filename, dpi=300, bbox_inches="tight")
    plt.close()


In [ ]:
shap.initjs()

def run_rf_train_test(dataframes, param_grid, eval_metrics, outer_reps=50, k=5, CV_reps=5, model_choice_metric='f1', 
                      res_dir=f"./results/", model_type='xgb', test_set=0.3, resampling=None, permutation=False):

    timestamp = int(time.time())
    res_dir = f"{res_dir}/{timestamp}_{model_type}_outer{outer_reps}_cvrep{CV_reps}_k{k}_{model_choice_metric}_testsize{test_set}_resampling{resampling}_perm{permutation}/"
    os.makedirs(res_dir, exist_ok=True)
    
    keys = list(dataframes.keys())

    # combine data categories
    combinations_keys = list(chain.from_iterable(combinations(keys, r) for r in range(1, 3)))
    combo_validation_scores = {}
    combo_test_scores = {}
    best_models = {} 
    best_shap_vals = {}
    best_paramses = {}

    all_val_scores = {}
    all_test_scores = {}
    all_models_sub = []

    for combo in tqdm(combinations_keys):
        validation_scores = {metric: [] for metric in eval_metrics}
        test_scores = {metric: [] for metric in eval_metrics}
        merged_df = dataframes[combo[0]].copy()
        top_models_group_sub = []
        
        for key in combo[1:]:
            merged_df = merged_df.merge(dataframes[key].copy(), how='inner', on=['id', TARGET_VAR])
        if TARGET_VAR not in merged_df.columns:
            raise ValueError(f"Target variable '{TARGET_VAR}' not found in merged dataframe for combo: {combo}")
    
        all_shap_values = []
        all_test_data = []
        best_overall_score = -np.inf 
        best_model_for_combo = None
        best_params_for_combo = None
        best_shap_for_combo = None

        for _ in range(outer_reps): # i repetitions of train/test

            # Prepare train/test split for this i (random & stratified)
            X_data, Y_data, X_test, Y_test = prepare_features_and_targets(merged_df.copy(), test_set=test_set, resampling=resampling)

            # Shuffle labels for permutation tests
            if permutation:
                Y_data = Y_data.sample(frac=1, random_state=None).reset_index(drop=True)
                Y_test = Y_test.sample(frac=1, random_state=None).reset_index(drop=True)

            else:
                best_model, best_scores, best_params = random_forest_kfold_grid_search(X_data, Y_data, 
                                                                                    param_grid, k=k, 
                                                                                    CV_reps=CV_reps, 
                                                                                    eval_metric=eval_metrics,
                                                                                    model_choice_metric=model_choice_metric,
                                                                                    res_dir=res_dir,
                                                                                    model_type=model_type,
                                                                                    combo=combo)
            # Collect metrics
            for metric, score in best_scores.items():
                validation_scores[metric].append(score)

            # Retrain the best model on the full training dataset and evaluate on the test set
            best_model.fit(X_data, Y_data)
            test_predictions = best_model.predict(X_test)
            proba_predictions = best_model.predict_proba(X_test)[:, 1]

            explainer = shap.TreeExplainer(best_model)
            shap_values = explainer.shap_values(X_test) 
            shap_values = shap_values[:, :, 1]

            # Append SHAP values and test data for later aggregation
            all_shap_values.append(shap_values)
            all_test_data.append(pd.DataFrame(X_test))

            if best_scores[model_choice_metric] > best_overall_score:
                best_overall_score = best_scores[model_choice_metric]
                best_model_for_combo = best_model
                best_params_for_combo = best_params
                best_shap_for_combo = shap_values  # Store SHAP values if needed

            if combo == ('group_sub',):
                top_models_group_sub.append((best_scores[model_choice_metric], deepcopy(best_model)))

            # Calculate and append metrics for the test set
            test_scores = compute_test_metrics(Y_test, test_predictions, proba_predictions, test_scores)

        # Keep track of the best model based on the model_choice_metric
        if combo not in best_models or best_scores[model_choice_metric] > combo_validation_scores[combo][model_choice_metric]['mean']:
            best_models[combo] = best_model_for_combo
            joblib.dump(best_model_for_combo, f"{res_dir}/model_{'_'.join(combo)}.joblib")

            best_shap_vals[combo] = best_shap_for_combo
            best_paramses[combo] = best_params_for_combo

            # Save top 10 models for group_sub combo
            if combo == ('group_sub',):
                top_models_group_sub = locals().get("top_models_group_sub", [])
                top_models_group_sub.append((best_overall_score, deepcopy(best_model_for_combo)))

                # Sort and save top 10 by score
                top_models_group_sub.sort(key=lambda x: x[0], reverse=True)
                top10 = top_models_group_sub[:10]

                subdir = os.path.join(res_dir, "top10_group_sub_models")
                os.makedirs(subdir, exist_ok=True)

                for i, (score, model) in enumerate(top10):
                    joblib.dump(model, f"{subdir}/model_rank{i+1}_score{score:.4f}.joblib")

                # Store back in locals so it's not overwritten each time
                locals()["top_models_group_sub"] = top_models_group_sub

        top2_features = plot_shap_summary_with_percentages(all_shap_values, all_test_data, res_dir, combo)

        plot_pdp_across_runs(
            best_model=best_model_for_combo,
            res_dir=res_dir,
            all_test_data=all_test_data,
            interaction_pair=tuple(top2_features)
        )

        # Calculate mean and 95% CI for validation scores
        z = norm.ppf(0.975)  # 95% confidence level
        final_validation_scores = {}
        for metric, scores in validation_scores.items():
            mean_score = np.mean(scores)
            std_error = np.std(scores, ddof=1) / np.sqrt(len(scores))
            ci_lower = mean_score - z * std_error
            ci_upper = mean_score + z * std_error
            final_validation_scores[metric] = {
                'mean': mean_score,
                '95%_CI': (ci_lower, ci_upper)
            }
        combo_validation_scores[combo] = final_validation_scores
        all_val_scores[combo] = validation_scores
        save_metrics_to_csv(all_val_scores, res_dir, 'all_val_scores.csv')

        # Calculate mean and 95% CI for test scores
        final_test_scores = {}
        for metric, scores in test_scores.items():
            mean_score = np.mean(scores)
            std_error = np.std(scores, ddof=1) / np.sqrt(len(scores))
            ci_lower = mean_score - z * std_error
            ci_upper = mean_score + z * std_error
            final_test_scores[metric] = {
                'mean': mean_score,
                '95%_CI': (ci_lower, ci_upper)
            }
        combo_test_scores[combo] = final_test_scores
        all_test_scores[combo] = test_scores
        save_metrics_to_csv(all_test_scores, res_dir, 'all_test_scores.csv')

        # For validation scores
        df_val = flatten_score_dict(combo_validation_scores, res_dir=res_dir, filename="validation_scores.csv")
        # For test scores
        df_test = flatten_score_dict(combo_test_scores, res_dir=res_dir, filename="test_scores.csv")
        
    return 

## Run Analysis

In [ ]:
dataframes = {
    'group_sub': b2_group_subjective_response,
}

In [ ]:
param_grid = {
    "n_estimators": [50],
    "max_depth": [3, 5],
    "min_samples_split": [2, 4, 8],
    "min_samples_leaf": [2, 3, 5]
}

eval_metrics = ['auc', 'f1', 'accuracy', 'specificity', 'sensitivity', 'PPV', 'NPV', 'MCC', 'balancedAcc', 'pr_auc', 'tn', 'fn', 'tp', 'fp']

##### Normal Run

In [ ]:
run_rf_train_test(
    dataframes=dataframes,
    param_grid=param_grid,
    eval_metrics=eval_metrics,
    outer_reps=2, # reduce for faster run --> this affects the results
    k=3,
    CV_reps=5,
    model_choice_metric='auc',
    res_dir="../results/negative_control",
    model_type='rf',
    test_set=0.3,
    resampling=None,
    permutation=False
)